# KSA: Polar Observations and Modelling

## Week X, Class 2 - Hydrographic sections

Now that we know how to build and plot hydrographic cross sections, lets look at what has happened with the Antarctic Bottom Water layer in the Australian basin.

If you are curious about this water mass and why do we care, check out [link](link).

In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cft
import cmocean as cm
import glob
import gsw
import matplotlib.pyplot as plt
import metpy
import numpy as np
import xarray as xr